In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import GlobalMaxPooling2D, Input

import pickle

from sklearn.neighbors import NearestNeighbors
from PIL import Image

In [2]:
#Dataset from kaggle https://www.kaggle.com/datasets/vikashrajluhaniwal/fashion-images

f_vectors = pickle.load(open('f_vectors.pkl', 'rb'))
f_vectors = np.array(f_vectors)
file_names = pickle.load(open('filenames.pkl', 'rb'))

In [3]:
feature_model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False,
                                                       input_shape=(224,224,3))
feature_model.trainable = False

preprocess_layer = tf.keras.applications.resnet50.preprocess_input

inputs = Input(shape=(224,224,3))
x = preprocess_layer(inputs)
x = feature_model(x)
output = GlobalMaxPooling2D()(x)

model = tf.keras.Model(inputs, output)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [11]:
def extract(img_path, model):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img = np.array(img.getdata()).reshape(img.size[0], img.size[1], 3)
    img = np.expand_dims(img, 0)
    pred = model.predict(img, verbose=0).flatten()
    return pred/np.linalg.norm(pred)

In [12]:
def recommend(image_path, top_k=5):
    result = extract(image_path, model)

    neighbours = NearestNeighbors(n_neighbors=top_k, algorithm='brute', metric='euclidean')
    neighbours.fit(f_vectors)

    distance, indices = neighbours.kneighbors([result])

    nearest_images = [file_names[i] for i in indices[0][1:]]
    original_image = image_path

    return [original_image] + nearest_images

def display_images(images_path):
    original = cv2.imread(images_path[0])
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    original_res = cv2.resize(original, (200, 200))
    images = images_path[1:]
    images = [cv2.imread(f'Dataset/{image}') for image in images]
    images = [cv2.resize(image, (200, 200)) for image in images]
    images = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in images]

    
    plt.figure(figsize=(4,4))
    plt.imshow(original_res)
    plt.title('Original Image', fontsize=10)
    plt.axis('off')
    # plt.show()
    
    fig, ax = plt.subplots(1, len(images), figsize=(17, 5))
    plt.suptitle('Recommendations', fontsize=15)
    for i, image in enumerate(images):
        ax[i].imshow(image)
        ax[i].axis('off')
    plt.show()


In [13]:
image_path = '5405.jpg'

recommendations = recommend(image_path, top_k=5)
display_images(recommendations)

2023-08-01 09:09:56.452428: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
[ WARN:0@442.132] global loadsave.cpp:244 findDecoder imread_('Dataset/5402.jpg'): can't open/read file: check file path/integrity
[ WARN:0@442.132] global loadsave.cpp:244 findDecoder imread_('Dataset/5404.jpg'): can't open/read file: check file path/integrity
[ WARN:0@442.133] global loadsave.cpp:244 findDecoder imread_('Dataset/5403.jpg'): can't open/read file: check file path/integrity
[ WARN:0@442.133] global loadsave.cpp:244 findDecoder imread_('Dataset/1831.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
